In [1]:
from util import labeledImage, normalize_coordinates, convert_to_yolo_format
from sklearn.model_selection import train_test_split
import os, shutil, yaml

In [2]:
source_images_dir = 'C:/Users/thoma/Desktop/Hackweek/qpm/images/'
source_labels_dir = 'C:/Users/thoma/Desktop/Hackweek/qpm/labels/'

labeled_images = []
#tag = ['DEFECT', 'ROI']
tag = 'CELLS'

for file in os.listdir(source_images_dir):
    # find all PNG file and it's ImageJ label
    if file.endswith(".tif"):
        image_path = os.path.join(source_images_dir, file)
        label_path = os.path.join(source_labels_dir, file.split('.')[0] + '.txt')
        labeled_images.append(labeledImage(image_path))
        labeled_images[-1].add_labels_from_file(tag, label_path)

In [3]:
train_and_val_set, test_set = train_test_split(labeled_images, test_size=0.1)
train_set, val_set = train_test_split(train_and_val_set, test_size=(2/9))

len(train_set), len(val_set), len(test_set)

(70, 20, 11)

In [4]:
# Create the molecule_images directory if it doesn't exist
output_dir = './train_images_qpm'
if not os.path.exists(output_dir): os.mkdir(output_dir)

train_dir = os.path.join(output_dir, 'train') 
val_dir   = os.path.join(output_dir, 'val') 
test_dir  = os.path.join(output_dir, 'test') 

# Create the sub-directories
for d in [train_dir, val_dir, test_dir]:
    if not os.path.exists(d): os.mkdir(d)
    
    images_sub_dir = os.path.join(d, 'images')
    labels_sub_dir = os.path.join(d, 'labels')
    
    for sub_dir in [images_sub_dir, labels_sub_dir]:
        if not os.path.exists(sub_dir): os.mkdir(sub_dir)

In [5]:
# make unified yolo tags 
tags = [tag]

# zip the dataset
dataset = [(train_dir, train_set),(val_dir, val_set),(test_dir, test_set)]

for d, s in dataset:
    images_sub_dir = os.path.join(d, 'images')
    labels_sub_dir = os.path.join(d, 'labels')

    # copy over the images
    for img in s:
        shutil.copyfile(img.path, os.path.join(images_sub_dir, img.name))
    
    # covert ImageJ labels to yolo format and save it to labels_sub_dir
    convert_to_yolo_format(s, labels_sub_dir, tags)

successfully generated labels for image  phase2.tif
successfully generated labels for image  phase44.tif
successfully generated labels for image  phase58.tif
successfully generated labels for image  phase26.tif
successfully generated labels for image  phase39.tif
successfully generated labels for image  phase20.tif
successfully generated labels for image  phase73.tif
successfully generated labels for image  phase33.tif
successfully generated labels for image  phase88.tif
successfully generated labels for image  phase6.tif
successfully generated labels for image  phase15.tif
successfully generated labels for image  phase92.tif
successfully generated labels for image  phase4.tif
successfully generated labels for image  phase43.tif
successfully generated labels for image  phase30.tif
successfully generated labels for image  phase35.tif
successfully generated labels for image  phase29.tif
successfully generated labels for image  phase18.tif
successfully generated labels for image  phase76.

In [6]:
# generate yolo yaml file
yolo_yaml = os.path.join(output_dir, 'cell_detection_yolov5.yaml')

with open(yolo_yaml, 'w') as yamlout:
    yaml.dump(
        {'train': train_dir,
         'val': val_dir,
         'nc': len(tags),
         'names': tags},
        yamlout,
        default_flow_style=None,
        sort_keys=False
    )

In [1]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies

In [2]:
yolov5_env = Environment(name="yolov5_env")

yolov5_env.docker.base_image  = "mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.0.3-cudnn8-ubuntu18.04"

conda_dep = CondaDependencies()
conda_dep.add_conda_package('python=3.8')

# install all the yolov5 requirement at the image build time
with open('./requirements.txt', 'r') as f:
    line = f.readline()
    
    while line != '':        
        if line.startswith('#') or len(line.split()) == 0:
            line = f.readline()
            continue
        
        conda_dep.add_pip_package(line.split()[0])
        line = f.readline()

yolov5_env.python.conda_dependencies=conda_dep

In [3]:
yolov5_env.get_image_details

<bound method Environment.get_image_details of {
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.0.3-cudnn8-ubuntu18.04",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "yolov5_env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
        

In [3]:
subscription_id = 'cfa7347c-f784-4a8a-b07f-e7f4b066c74c'
resource_group  = 'rg-amlclass-thro1'
workspace_name  = 'aml-amlclass-thro1'
ws = Workspace(subscription_id, resource_group, workspace_name)

experiment = Experiment(workspace=ws, name='cell_detection_yolo_training')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [4]:
config = ScriptRunConfig(source_directory='./deploy_yolo_training/',   ## folder in which the script is located
                         script='training_on_aml.py',       ## script name
                         compute_target='GPU-thro1',
                         environment=yolov5_env) 

In [5]:
%pwd

'C:\\Users\\thoma\\Desktop\\Hackweek'

In [6]:
run = experiment.submit(config)
aml_url = run.get_portal_url()
print(aml_url)

https://ml.azure.com/runs/cell_detection_yolo_training_1621571157_3e3246a4?wsid=/subscriptions/cfa7347c-f784-4a8a-b07f-e7f4b066c74c/resourcegroups/rg-amlclass-thro1/workspaces/aml-amlclass-thro1&tid=f6b6dd5b-f02f-441a-99a0-162ac5060bd2
